# Descrição do programa

## Visão Geral

Este script realiza uma análise de risco para um fundo de investimento composto por ações listadas na B3. Ele automatiza a coleta de dados históricos de preços utilizando a API do Yahoo Finance, abrange um período de cinco anos, e calcula indicadores financeiros e de risco relevantes, como volatilidade, Value at Risk (VaR), Conditional Value at Risk (CVaR), beta via regressão, e índices de Sharpe e Treynor. O código também permite avaliar o risco sob múltiplos níveis de confiança, proporcionando uma visão abrangente da exposição do fundo a diferentes cenários de mercado.

## Autor

João Vitor Figueiredo Villa -[LinkedIn](https://www.linkedin.com/in/jo%C3%A3o-vitor-figueiredo-villa-8323b8323/)

# Instalação e Implementação dos Pacotes

In [79]:
%pip install pandas
%pip install numpy
%pip install statsmodels
%pip install yfinance
%pip install scipy
%pip install python-bcb
%pip install datetime
%pip install python-dateutil
%pip install statsmodels

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


...


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import statsmodels
from scipy.stats import norm
from datetime import datetime
from dateutil.relativedelta import relativedelta
import statsmodels.api as sm

from bcb import sgs
import yfinance as yf

# Dados Iniciais para análise

Importando arquivo das ações

In [ ]:
# Varíaveis de data que permite, que o código tenha series temporais fluídas e que se atualizam sempre que o código é rodado
agora = datetime.now()
date5y = agora - relativedelta(years=5)

agora = agora.strftime("%Y-%m-%d")
date5y = date5y.strftime("%Y-%m-%d")


#Açoes usadas como exemplo para testar a testar o código para as fórmulas de análise de risco
a1 = yf.Ticker("PETR4.SA").history(start=date5y,end=agora)
a2 = yf.Ticker("JBSS3.SA").history(start=date5y,end=agora)
a3 = yf.Ticker("BBAS3.SA").history(start=date5y,end=agora)
a4 = yf.Ticker("VIVT3.SA").history(start=date5y,end=agora)
a5 = yf.Ticker("ITUB4.SA").history(start=date5y,end=agora)

listaAcoes = [a1,a2,a3,a4,a5]

Variáveis Básicas

In [4]:
#retornos logarítmicos diários
lRetD = []
#variância dos retornos logarítmicos diários
lVarc = []
for i0 in listaAcoes:
    close_prices = i0['Close']
    returns = np.log(close_prices / close_prices.shift(1)).dropna()
    
    lRetD.append(returns)
    lVarc.append(np.var(returns))

#z1 é para um nível de confiança de 95% (Primeira análise)
a1 = 0.95
z1 = norm.ppf(a1)
dz1 = norm.pdf(z1)

#z2 é para um nível de confiança de 99% (Segunda análise)
a2 = 0.99
z2 = norm.ppf(a2)
dz2 = norm.pdf(z2)

valInv = 1000000
lPorc = [0.2,0.2,0.20,0.20,0.20]
lInv = []
for i1 in lPorc:
    lInv.append(i1*valInv)

# 1) Análise Individual dos Investimentos

## Volatilidade

In [49]:
#quanto menor a volatilidade, menor é a oscilação do ativo
lDP = []
for i2 in lVarc:
    dp = np.sqrt(i2)
    lDP.append(dp)

print("Volatilidade das ações:")
for n0,i3 in enumerate(lDP):
    print(f"Ação {n0} -> {i3:.4f}")

Volatilidade das ações:
Ação 0 -> 0.0206
Ação 1 -> 0.0213
Ação 2 -> 0.0162
Ação 3 -> 0.0145
Ação 4 -> 0.0142


## Value at Risk (VaR)

valor limite de perdas

- Para 95% de nível de confiança

In [ ]:

# a perda não ultrapassa o valor do VaR, a perda vai ser em 95% das vezes menor ou igual a esse valor 
# valor limite de perdas

lVaR1 = []
for n1,i4 in enumerate(lDP):
    valAtR = z1*i4*lInv[n1]
    lVaR1.append(valAtR)

print("VaR (Para 95% de nível de confiança):")
for n2,i5 in enumerate(lVaR1):
    print(f"Ação {n2} -> perda máxima esperada de R$ {i5:.2f} (Inv inicial = R$ {lInv[n2]:.2f})")
print("-="*20)

print(f"Perda máxima do fundo: R$ {np.sum(lVaR1):.2f}")

VaR (Para 95% de nível de confiança):
Ação 0 -> perda máxima esperada de R$ 6807.88 (Inv inicial = R$ 200000.00)
Ação 1 -> perda máxima esperada de R$ 7036.34 (Inv inicial = R$ 200000.00)
Ação 2 -> perda máxima esperada de R$ 5350.33 (Inv inicial = R$ 200000.00)
Ação 3 -> perda máxima esperada de R$ 4787.54 (Inv inicial = R$ 200000.00)
Ação 4 -> perda máxima esperada de R$ 4697.51 (Inv inicial = R$ 200000.00)
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Perda máxima do fundo: R$ 28679.60


- Para 99% de nível de confiança

In [ ]:
# a perda não ultrapassa o valor do VaR, a perda vai ser em 99% das vezes menor ou igual a esse valor (cenário de incerteza)

lVaR2 = []
for n3,i6 in enumerate(lDP):
    valAtR = z2*i6*lInv[n3]
    lVaR2.append(valAtR)

print("VaR (Para 99% de nível de confiança):")
for n4,i7 in enumerate(lVaR2):
    print(f"Ação {n4} -> perda máxima esperada de R${i7:.2f} (Inv inicial = R$ {lInv[n4]:.2f})")
print("-="*20)

print(f"Perda máxima esperada do fundo: R$ {np.sum(lVaR2):.2f}")

VaR (Para 99% de nível de confiança):
Ação 0 -> perda máxima esperada de R$9613.56 (Inv inicial = R$ 200000.00)
Ação 1 -> perda máxima esperada de R$9936.16 (Inv inicial = R$ 200000.00)
Ação 2 -> perda máxima esperada de R$7555.31 (Inv inicial = R$ 200000.00)
Ação 3 -> perda máxima esperada de R$6760.58 (Inv inicial = R$ 200000.00)
Ação 4 -> perda máxima esperada de R$6633.46 (Inv inicial = R$ 200000.00)
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Perda máxima esperada do fundo: R$ 40499.07


## Conditional Value at Risk (CVaR)

valores da cauda de distribuição de perdas em cenários de perda extrema (tende a ser maior do que o VaR)


- Para 95% de confiança

In [52]:
# média das perdas que ocorrem nos 5% piores cenários 

lCVaR1 = []
for t0,i8 in zip(lVaR1,lDP):
    cvar = i8 + (t0*dz1/(1-a1))
    lCVaR1.append(cvar)

print("CVaR (Para 95% de nível de confiança):")
for n4, i9 in enumerate(lCVaR1):
    print(f"Ação {n4} -> pior média de perdas esperada de R$ {i9:.2f} (VaR = R$ {lVaR1[n4]:.2f})")
print("-="*20)

print(f"Pior média de perdas do fundo esperado: R$ {np.sum(lCVaR1):.2f}")
    

CVaR (Para 95% de nível de confiança):
Ação 0 -> pior média de perdas esperada de R$ 13924.18 (VaR = R$ 6807.88)
Ação 1 -> pior média de perdas esperada de R$ 14391.43 (VaR = R$ 7036.34)
Ação 2 -> pior média de perdas esperada de R$ 10943.03 (VaR = R$ 5350.33)
Ação 3 -> pior média de perdas esperada de R$ 9791.96 (VaR = R$ 4787.54)
Ação 4 -> pior média de perdas esperada de R$ 9607.83 (VaR = R$ 4697.51)
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Pior média de perdas do fundo esperado: R$ 58658.43


- Para 99% de confiança

In [53]:
# média das perdas que ocorrem nos 1% piores cenários

lCVaR2 = []
for i10,t1 in zip(lVaR2,lDP):
    cvar = i10 + (t1*dz2/(1-a2))
    lCVaR2.append(cvar)

print("VaR (Para 99% de nível de confiança):")
for n5,i11 in enumerate(lCVaR2):
    print(f"Ação {n5} -> pior média de perdas de R$ {i11:.2f} (VaR = R$ {lVaR2[n5]:.2f})")
print("-="*20)

print(f"Pior média de perdas do fundo: R$ {np.sum(lCVaR2):.2f}")

VaR (Para 99% de nível de confiança):
Ação 0 -> pior média de perdas de R$ 9613.61 (VaR = R$ 9613.56)
Ação 1 -> pior média de perdas de R$ 9936.22 (VaR = R$ 9936.16)
Ação 2 -> pior média de perdas de R$ 7555.35 (VaR = R$ 7555.31)
Ação 3 -> pior média de perdas de R$ 6760.62 (VaR = R$ 6760.58)
Ação 4 -> pior média de perdas de R$ 6633.50 (VaR = R$ 6633.46)
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Pior média de perdas do fundo: R$ 40499.30


# 2) Análise Comparativa

## Indice de Sharpe

- menor que 0 = Retorno abaixo da tx livre de risco (mau desempenho ajustado ao risco)
- 0 a 1: Retorno baixo em relação ao risco
- 1 a 1.99: Bom (retorno razoável para o rísco assumido)
- 2 a 2.99: Muit Bom (portfólio eficiente)
- maior ou igual a 3: Excelente (alto retorno com risco controlado)

In [130]:
acoesRD = pd.concat([pd.Series(serie, name=f'Acao {i}') for i, serie in enumerate(lRetD)], axis=1)
print(acoesRD)

pesos = np.array(lPorc)
matrixCov = acoesRD.cov()
volPortf = np.sqrt(np.dot(pesos.T, np.dot(matrixCov, pesos)))

cdiAnual = sgs.get(1178,last=1)
cdiAnual = (cdiAnual.iloc[0,0])/100

cdiDia = (1 + cdiAnual)**(1/252) - 1

mediaRetDia = np.dot(acoesRD.mean(), pesos)

sharpeF =  (mediaRetDia - cdiDia)/ volPortf
print(f"Índice de Sharpe do portfólio: {sharpeF:.4f}")

                             Acao 0    Acao 1    Acao 2    Acao 3    Acao 4
Date                                                                       
2020-05-25 00:00:00-03:00  0.042470  0.006276  0.099733  0.018188  0.042633
2020-05-26 00:00:00-03:00  0.009706 -0.005308 -0.022797  0.008458 -0.040457
2020-05-27 00:00:00-03:00  0.013131  0.051390  0.003566 -0.002262  0.030811
2020-05-28 00:00:00-03:00 -0.008060  0.010969 -0.003566 -0.010972 -0.017858
2020-05-29 00:00:00-03:00  0.028424 -0.001365  0.001623 -0.005875 -0.011650
...                             ...       ...       ...       ...       ...
2025-05-15 00:00:00-03:00 -0.001254 -0.004330 -0.012171 -0.008842  0.013358
2025-05-16 00:00:00-03:00  0.004696 -0.007172 -0.135672 -0.012152  0.005822
2025-05-19 00:00:00-03:00 -0.001250  0.030133 -0.024848 -0.005408  0.011803
2025-05-20 00:00:00-03:00  0.004057  0.046781  0.018204  0.002167 -0.001305
2025-05-21 00:00:00-03:00 -0.011275  0.001665 -0.009456 -0.007605 -0.020309

[1242 rows 

## Beta e Alpha do fundo

- está sendo usado o Ibovespa como benchmark

- Alpha > 0: portfólio está gerando retorno acima do esperado pelo CAPM
- Alpha < 0: portfólio está gerando retorno abaixo do esperado pelo CAPM

O beta é a proporção que o fundo se movimenta com o benckmark

- Beta < 0: Inversamente correlacionado (o fundo anda no sentido oposto ao mercado)
- Beta = 0: Independente do mercado (o fundo não se movimenta com o mercado)
- Beta entre 0 e 1: Menos volátil que o mercado (defensivo, segue o mercado com menos intensidade)
- Beta = 1: Neutro ao mercado (o fundo sobe ou cai na mesma proporção do benchmark)
- Beta > 1: Mais volátil que o mercado (agressivo, amplifica os movimentos do mercado)

In [139]:
retornoFundoDia = pd.Series(acoesRD.dot(pesos), index=acoesRD.index)

ibov5y = yf.Ticker("^BVSP").history(start=date5y,end=agora)
ibov5y = ibov5y['Close']

retIbovDia = np.log(ibov5y / ibov5y.shift(1)).dropna()
medIbovDia = np.mean(retIbovDia)

dfAcoesBt = pd.merge(retIbovDia.to_frame('ibov'), 
                     retornoFundoDia.to_frame('fundo'), 
                     left_index=True, right_index=True, 
                     how='inner')
dfAcoesBt = dfAcoesBt.dropna()

X = sm.add_constant(dfAcoesBt['ibov'])
y = dfAcoesBt['fundo']

modeloRL = sm.OLS(y, X).fit()
print(modeloRL)

beta = modeloRL.params['ibov']
alpha = modeloRL.params['const']

print("-="*20)
print(f"Beta: {beta:.4f}")
print(f"Alpha: {alpha:.4f}")



-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Beta: 0.8554
Alpha: 0.0005


## Indice de Treynor

- Positivo e alto = O fundo está entregando bom retorno ajustado ao risco de mercado (quanto maior, melhor)
- Próximo de zero = O fundo não gera retorno além da taxa livre de risco, mesmo assumindo risco de mercado
- Negativo = O fundo teve desempenho pior que a taxa livre de risco, mesmo assumindo risco (performance ruim)

In [138]:
print("-="*20)

treynorD =  (mediaRetDia - medIbovDia)/beta
print(f"Índice de Treynor diário: {treynorD:.6f}")

retFundoAn = (1+mediaRetDia)**252 - 1
medIbovAn = (1 + medIbovDia)**252 - 1
treynorA = (retFundoAn - medIbovAn)/beta
print("-="*20)
print(f"Índice de Treynor diário: {treynorA:.6f}")

-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Índice de Treynor diário: 0.000457
-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=
Índice de Treynor diário: 0.134197
